In [ ]:
pip install tabulate


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('your_dataset_path.csv')

# Filter for the 2023 draft class
df_2023 = df[df['Draft Year'] == 2023]

# Handling NaN values and reordering based on advanced stats
df_2023['WS_Reordering'] = df_2023['WS'].rank(method='max', ascending=False, na_option='keep')
df_2023['WS48_Reordering'] = df_2023['WS/48'].rank(method='max', ascending=False, na_option='keep')
df_2023['BPM_Reordering'] = df_2023['BPM'].rank(method='max', ascending=False, na_option='keep')
df_2023['VORP_Reordering'] = df_2023['VORP'].rank(method='max', ascending=False, na_option='keep')

# Preparing the final output
final_output = df_2023[['Pk', 'Tm', 'Player', 'WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].copy()
final_output.rename(columns={'Pk': 'Pick', 'Tm': 'Team'}, inplace=True)
final_output = final_output.sort_values('Pick').reset_index(drop=True)

# Export the final output to CSV
final_output.to_csv('2023_NBA_Draft_Reordering_Full.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.stats import spearmanr
from tabulate import tabulate

# Load the dataset
df = pd.read_csv('/content/Testing DRAFT CLASSES.csv', encoding='utf-8')
df.fillna(-100, inplace=True)

# Create an empty list to store results
results = []

# Loop through each year from 1984 to 2023
for year in range(1984, 2024):
    # Filter for the specific draft year and make a copy
    df_year = df[df['Draft Year'] == year].copy()

    # Handling NaN values and reordering based on advanced stats
    df_year['WS_Reordering'] = df_year['WS'].rank(method='max', ascending=False, na_option='keep')
    df_year['WS48_Reordering'] = df_year['WS/48'].rank(method='max', ascending=False, na_option='keep')
    df_year['BPM_Reordering'] = df_year['BPM'].rank(method='max', ascending=False, na_option='keep')
    df_year['VORP_Reordering'] = df_year['VORP'].rank(method='max', ascending=False, na_option='keep')

    # Calculate 'Average Rank'
    df_year['Average_Rank'] = df_year[['WS_Reordering', 'WS48_Reordering', 'BPM_Reordering', 'VORP_Reordering']].mean(axis=1)

    # Rename columns to ensure we have 'Pick' and 'Team' instead of 'Pk' and 'Tm'
    final_output = df_year.rename(columns={'Pk': 'Pick', 'Tm': 'Team'})

    # Calculate the Spearman rank correlations to see how well the draft order matches the reordered rankings
    correlations = [
        spearmanr(final_output['Pick'], final_output['WS_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['WS48_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['BPM_Reordering'].fillna(-100)).correlation,
        spearmanr(final_output['Pick'], final_output['VORP_Reordering'].fillna(-100)).correlation
    ]
    spearman_corr_avg_rank = spearmanr(final_output['Pick'], final_output['Average_Rank']).correlation
    average_spearman = np.nanmean(correlations)  # Using nanmean to handle potential NaNs in correlations

    # Append results to the list
    results.append([year, average_spearman, spearman_corr_avg_rank])

# Displaying the results in a formatted table
results_df = pd.DataFrame(results, columns=['Year', 'Average Spearman Correlation', 'Spearman Correlation (Original vs. Average Rank)'])

# Assign colors based on Spearman correlation values
def get_color(value):
    # Define colormap ranging from white to light blue
    cmap = plt.cm.get_cmap('Blues')
    norm = mcolors.Normalize(-1, 1)  # Normalize values to range from -1 to 1
    return mcolors.to_hex(cmap(norm(value)))

# Apply color formatting to the results DataFrame
results_df['Average Spearman Color'] = results_df['Average Spearman Correlation'].apply(get_color)
results_df['Spearman Correlation Color'] = results_df['Spearman Correlation (Original vs. Average Rank)'].apply(get_color)

# Truncate numerical values to 3 decimal places with zero-padding
results_df['Average Spearman Correlation'] = results_df['Average Spearman Correlation'].apply(lambda x: f"{x:.3f}")
results_df['Spearman Correlation (Original vs. Average Rank)'] = results_df['Spearman Correlation (Original vs. Average Rank)'].apply(lambda x: f"{x:.3f}")

# Save results to a CSV file
results_df.to_csv('spearman_correlation_results_with_colors.csv', index=False)

# Print the tabulated results with color values
print(tabulate(results_df[['Year', 'Average Spearman Correlation', 'Average Spearman Color', 'Spearman Correlation (Original vs. Average Rank)', 'Spearman Correlation Color']], headers='keys', tablefmt='pretty'))


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('/content/Spearman Coefficients with Colors (1984 to 2023).csv')

# Start the LaTeX table code
latex_code = "\\begin{tabular}{|l|c|c|}\n\\hline\n"
latex_code += "\\rowcolor{gray!20}\n\\textbf{Year} & \\textbf{Average Spearman Corr.} & \\textbf{Spearman Corr. (Avg. Rank)} \\\\ \\hline\n"

# Loop through the DataFrame and add rows to the LaTeX code
for index, row in df.iterrows():
    year = int(row['Year'])  # Convert year to integer
    avg_corr = row['Average Spearman Correlation']
    rank_corr = row['Spearman Correlation (Average Rank)']
    # Check the values to determine the color of the cell
    avg_corr_color = 'lightgreen' if avg_corr >= 0.5 else 'lightred'
    rank_corr_color = 'lightgreen' if rank_corr >= 0.5 else 'lightred'
    # Add the row to the LaTeX code
    latex_code += f"\\textbf{{{year}}} & \\cellcolor{{{avg_corr_color}}}{avg_corr:.3f} & \\cellcolor{{{rank_corr_color}}}{rank_corr:.3f} \\\\ \\hline\n"

# End the LaTeX table code
latex_code += "\\end{tabular}\n"

# Save the LaTeX code to a .txt file
with open('spearman_coefficients_table.txt', 'w') as file:
    file.write(latex_code)

print("LaTeX code saved as spearman_coefficients_table.txt")
